In [1]:
#0. conda activate mip2019, 라이브러리 로드
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [21]:
#1. 데이터 불러오기
dataFrame = pd.read_csv("NHIS_OPEN_GJ_2016.csv", engine = "python")
print(dataFrame.shape)
print(dataFrame.columns)

(1000000, 34)
Index(['���س⵵', '�������Ϸù�ȣ', '�����ڵ�', '���ɴ��ڵ�(5������)', '�õ��ڵ�',
       '����(5Cm����)', 'ü��(5Kg����)', '�㸮�ѷ�', '�÷�(��)', '�÷�(��).1',
       'û��(��)', 'û��(��).1', '���������', '�̿ϱ�����', '��������(��������)',
       '���ݷ����׷�', 'Ʈ���۸������̵�', 'HDL�ݷ����׷�', 'LDL�ݷ����׷�', '������',
       '��ܹ�', '��ûũ����Ƽ��', '(��û����Ƽ)AST', '(��û����Ƽ)ALT', '������Ƽ��',
       '��������', '���ֿ���', '�������� ���˿���', 'ġ�ƿ��������', '���ġ����',
       'ġ�Ƹ���������', '��3�뱸ġ(�����)�̻�', 'ġ��', '�����Ͱ�������'],
      dtype='object')


In [22]:
dataFrame[:5]

,���س⵵,�������Ϸù�ȣ,�����ڵ�,���ɴ��ڵ�(5������),�õ��ڵ�,����(5Cm����),ü��(5Kg����),�㸮�ѷ�,�÷�(��),�÷�(��).1,...,������Ƽ��,��������,���ֿ���,�������� ���˿���,ġ�ƿ��������,���ġ����,ġ�Ƹ���������,��3�뱸ġ(�����)�̻�,ġ��,�����Ͱ�������
0,2016,1,1,10,27,175.0,75.0,86.0,1.0,1.2,...,45.0,3.0,NaN,0,NaN,NaN,NaN,NaN,NaN,20171219
1,2016,2,2,11,41,160.0,65.0,85.0,1.5,1.2,...,19.0,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN,20171219
2,2016,3,2,11,43,150.0,55.0,80.0,1.5,1.5,...,64.0,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN,20171219
3,2016,4,1,12,48,175.0,70.0,85.0,1.2,1.2,...,69.0,2.0,NaN,1,NaN,NaN,NaN,NaN,1.0,20171219
4,2016,5,2,6,11,160.0,50.0,67.0,0.4,0.5,...,20.0,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN,20171219


In [6]:
#2. 데이터 샘플링 및 시각화하기.

#특정 열(키, 체중) 선택
twoCols = dataFrame.loc[:, ["신장(5cm단위)", "체중(5kg단위)"]]
#키와 몸무게 중에 NaN값이 있는 row 제거
twoCols = twoCols.dropna()

#샘플링
samples = twoCols.sample(10)
samples

#시각화1
hei = list(samples.iloc[:,0])
print(hei)
wei = list(samples/iloc[:,1])
print(wei)

plt.scatter(hei, wei)
plt.show()

#시각화2(개수가 많을수록 원의 반지름 크게)
samples = twoCols.sample(10000)
grouped = samples.groupby(["신장(5cm단위)", "체중(5kg단위)"]).size() #[신장, 체중]pair 별 개수
print(grouped)

plt.scatter(x = grouped.index.get_level_values(0), y=grouped.index.get_level_values(1), s=[10*1.005**n for n in grouped.values])

KeyError: "None of [Index(['신장(5cm단위)', '체중(5kg단위)'], dtype='object')] are in the [columns]"

In [ ]:
#3. 학습 모델 정의하기 X(키) -> Y(체중)
tf.reset_default_graph()
#데이터셋이 들어갈 placeholder 정의
X = tf.placeholder(tf.float32, name='X')
Y = tf.placeholder(tf.float32, name='Y')

#선형회귀 모델 학습에 사용될 변수 정의 (학습되며 값이 바뀜)
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='W')
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='b')

#선형회귀 모델(가설) 정의
hypothesis = W*X + b

#로스함수 정의 (absolute error 사용)
loss = tf.reduce_mean(tf.abs(hypothesis - Y))

#최적화 방법 정의(확률적 경사 하강법 사용)
# *** learning rate 조절 ***
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)

train_op = optimizer.minimize(loss)

In [ ]:
#세션을 생성하고 학습 수행
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#학습 전 가중치 초기값
print(sess.run([W, b]))

for iteration in range(1001):
    # 전체 데이터에서 10000개씩 sampling하고, list로 변환함
    samples = twoCols.sample(10000)
    sample_x = samples.lloc[:. 0].values #키
    sample_y = samples.lloc[:. 1].values #체중
    
    _, _loss, predict = sess.run([train_op, loss, hypothesis], feed_dict={X:sample_x, Y : sample_y})
    
    if iteration % 50 == 0:
        print("=============== iter", str(iteraion), "=================")
        print(sess.run([W,b]))
        print("키", sample_x[:3])
        print("예측 체중", np.round(predict[:3], 1))
        print("정답 체중", sample_y[:3])
        print("loss", _loss)

In [ ]:
print(sess.run([W,b]))

height = 170
weight = sess.run(hypothesis, feed_dict = {X:[height]})
print(weight, "kg")

In [ ]:
threeCols = dataFrame.loc[:, ["신장(5Cm단위)", "체중(5Kg단위)", "허리둘레"]]

threeCols = threeCols.dropna()

threeCols.groupby("허리둘레").mean()

In [ ]:
tf.reset_default_graph()

X1 = tf.placeholder(tf.float32, name='X1')
X2 = tf.placeholder(tf.float32, name = 'X2')
Y = tf.placeholder(tf.float32, name = 'Y')

W1 = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name='W1')
W2 = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name = 'W2')
b1 = tf.Variable(tf.random_uniform([1], -1.0, 1.0), name = 'b1')

hypothesis = W1*X1 + W2*X2 +b1

loss = tf.reduce_mean(tf.abs(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
train_op = optimizer.minimize(loss)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for iteration in range(1001):
    samples = threeCols.sample(10000)
    sample_x1 = samples.iloc[:, 0].values
    sample_x2 = samples.iloc[:, 1].values
    sample_y = samples.iloc[:, 2].values
    
    _, _loss, predict = sess.run([train_op, loss, hypothesis], feed_dict={X1:sample_x1, X2:sample_x2, Y:sample_y})
    
    
    if iteration % 50 == 0:
        print("=================== iter", str(literation), "=====================")
        print(sess.run([W1, W2, b1]))
        print("키", sample_x1[:3])
        print("체중", sample_x2[:3])
        print("예측 허리둘레", np.round(predict[:3], 1))
        print("정답 허리둘레", sample_y[:3])
        print("loss", _loss)

In [ ]:
print(sess.run([W1, W2, b1]))

height = 170
weight = 65
waist = sess.run(hypothesis, feed_dict={X1:[height], X2:[weight]})

print("---허리둘레---")
print(waist, "cm")
print(waist/2.54, "인치")